# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
</div>
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
data.shape

(4870, 65)

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


**1. What test is appropriate for this problem? Does CLT apply?**

A two-sampled z statistic is appropriate as we have two groups of datasets measuring the same parameter (getting a call back or not). The central limit theorem (CLT) states that it is applicable to a distribution of sample size greater than 30 even if the population is not normally distributed. Therefore, the CLT is applicable for solving the problem. (we can also check if n*p and n * (1-p) are greater than 10. Both are obviously greater than 10 (taking the population of blacks and whites), therefore CLT applies)

**2. What are the null and alternate hypotheses?**

**Ho:** There is no difference in callback between white- and black-sounding names <br/>
**Ha:** There is a difference in callback between white- and black-sounding names 

**3. Compute margin of error, confidence interval, and p-value.**

In [5]:
Wtot = data[data.race=='w'].shape[0] 

In [6]:
Btot = data[data.race=='b'].shape[0] 

In [7]:
Bc = sum(data[data.race=='b'].call)
Bn = Btot - Bc 

In [8]:
Wc = sum(data[data.race=='w'].call) 
Wn = Wtot - Wc 

In [9]:
print("The total number of white sounding names is {}".format(Wtot))
print("The total number of black sounding names is {}".format(Btot))
print("The number of callbacks for white sounding names is {:3.0f}".format(Wc))
print("The number of callbacks for black sounding names is {:3.0f}".format(Bc))
print("The number of white sounding names which did not get a callback is {:3.0f}".format(Wn))
print("The number of black sounding names which did not get a callback is {:3.0f}".format(Bn))

The total number of white sounding names is 2435
The total number of black sounding names is 2435
The number of callbacks for white sounding names is 235
The number of callbacks for black sounding names is 157
The number of white sounding names which did not get a callback is 2200
The number of black sounding names which did not get a callback is 2278


A quick look at the data shows **black-sounding names got a callback of 157 out of 2435 and white-sounding names got a callback of 235 out of 2435**

In [10]:
Bc_mean = (1 * Bc + Bn * 0) / Wtot

In [11]:
Bc_var = (Bc_mean * (1 - Bc_mean))/Btot

In [12]:
Wc_mean = (1 * Wc + 0 * Wn) / Wtot

In [13]:
Wc_var = (Wc_mean * (1 - Wc_mean))/Wtot

In [14]:
print("The mean of callbacks for white sounding names is {:0.4f}".format(Wc_mean))
print("The mean of callbacks for black sounding names is {:0.4f}".format(Bc_mean))
print("The variance of callbacks for white sounding names is {:0.6f}".format(Wc_var))
print("The variance of callbacks for black sounding names is {:0.6f}".format(Bc_var))

The mean of callbacks for white sounding names is 0.0965
The mean of callbacks for black sounding names is 0.0645
The variance of callbacks for white sounding names is 0.000036
The variance of callbacks for black sounding names is 0.000025


In [15]:
Tc = Wc + Bc #Total of individuals who got a callback 
Ttot = Wtot + Btot #Total of individuals in the dataset
Tmean = Tc/Ttot

In [17]:
# we can evaluate the combined standard deviation by adding the variances for black and white callbacks or
Tvar = Wc_var + Bc_var
Tstd = np.sqrt(Tvar)
#calculate the variance from the mean of combined (white and black) callbacks
Tstd2 = np.sqrt(2 * 0.0805 * (1 - 0.0805) / Wtot)
Tstd2 = Tstd2 
Tstd2

0.0077972116323879789

In [17]:
Rdiff = Wc_mean - Bc_mean
z_score = (Rdiff - 0)/Tstd
CIu = 1.96 * Tstd
pval = stats.norm.sf(np.abs(z_score))*2

In [18]:
print("The mean of callbacks for both white and black sounding names is {:0.4f}".format(Tmean))
print("The standard deviation of callbacks for both white and black sounding names is {:0.4f}".format(Tstd))
print("The difference in mean of callbacks between white and black sounding names is {:0.4f}".format(Rdiff))
print("The z score associated with the calculated mean given the null hypothesis is {:0.4f}".format(z_score))
print("The upper bound for 95% confidence is {:0.4f}".format(Rdiff + CIu))
print("The lower bound for 95% confidence is {:0.4f}".format(Rdiff - CIu))
print("The p-value associated with the calculated z score given the null hypothesis is {:0.6f}".format(pval))

The mean of callbacks for both white and black sounding names is 0.0805
The standard deviation of callbacks for both white and black sounding names is 0.0078
The difference in mean of callbacks between white and black sounding names is 0.0320
The z score associated with the calculated mean given the null hypothesis is 4.1156
The upper bound for 95% confidence is 0.0473
The lower bound for 95% confidence is 0.0168
The p-value associated with the calculated z score given the null hypothesis is 0.000039


**The z_score, p-value and the confidence interval indicate that we should reject the null hypothesis**

** 4. Write a story describing the statistical significance in the context or the original problem.**

The results of the survey show that there is a statistically significant difference between white-sounding and black-sounding names to get a callback from prospective employees. There is a 95% chance that the difference between callbacks for white-sounding and black-sounding names lies in the interval [0.017,0.047]. The Type I error associated with rejecting the null hypothesis is 0.0039%.

** 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis? **

The analysis is limited since it is looking at race/name and callback rate. There are other factors that contribute in increased/decreased callback rate. I would have to look at other factors including non-work related (gender, disability, etc) and work-related (experience, work history, internships).